In [1]:
from sklearn.model_selection import KFold
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.externals import joblib

In [4]:
#Read data
df = pd.read_csv('fill.csv')

#For Volume Prediction
X_VOL = df[['Weight', 'PCV', 'PCV\ndonor', 'PCV_target']]
y_VOL = df['Volume']

#For PCV Predicion
X_PCV = df[['Weight', 'PCV', 'PCV\ndonor', 'Volume']]
y_PCV = df['PCV_afterdonation']



X_VOL = np.array(X_VOL)
y_VOL = np.array(y_VOL)

X_PCV = np.array(X_PCV)
y_PCV = np.array(y_PCV)

scaler_VOL = StandardScaler()
scaler_VOL.fit(X_VOL)
X_VOL = scaler_VOL.transform(X_VOL)

scaler_PCV = StandardScaler()
scaler_PCV.fit(X_PCV)
X_PCV = scaler_PCV.transform(X_PCV)

In [5]:
#Export scaler
joblib.dump(scaler_VOL, 'scaler_VOL.pkl')
joblib.dump(scaler_PCV, 'scaler_PCV.pkl')


['scaler_PCV.pkl']

In [6]:
#LR
model_VOL = LinearRegression()
model_VOL.fit(X_VOL,y_VOL)


model_PCV = LinearRegression()
model_PCV.fit(X_PCV,y_PCV)

joblib.dump(model_VOL, 'model_VOL_LR.pkl')
joblib.dump(model_PCV, 'model_PCV_LR.pkl')







['model_PCV_LR.pkl']

In [7]:
from sklearn import ensemble
optimal_estimator = [10,50,100,150,200,250,300]

MSE = 1000000
temp_var_vol = 10000
for i in optimal_estimator:
    kf = KFold(n_splits=10)
    kf.get_n_splits(X_VOL)
    
    local_MSE = []
    for train_index, test_index in kf.split(X_VOL):
        X_train, X_test = X_VOL[train_index], X_VOL[test_index]
        y_train, y_test = y_VOL[train_index], y_VOL[test_index]
        model = ensemble.GradientBoostingRegressor(random_state=1,max_depth=1,n_estimators=i)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        x_temp = (metrics.mean_squared_error(y_test, predictions))
        local_MSE.append(x_temp)
    x = np.mean(local_MSE)
    print(x)
    if x < MSE:
        MSE = x
        temp_var_vol = i
        
print(temp_var_vol)


MSE = 1000000
temp_var_pcv = 10000
for i in optimal_estimator:
    kf = KFold(n_splits=10)
    kf.get_n_splits(X_PCV)
    
    local_MSE = []
    for train_index, test_index in kf.split(X_PCV):
        X_train, X_test = X_PCV[train_index], X_VOL[test_index]
        y_train, y_test = y_PCV[train_index], y_PCV[test_index]
        model = ensemble.GradientBoostingRegressor(random_state=1,max_depth=1,n_estimators=i)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        x_temp = (metrics.mean_squared_error(y_test, predictions))
        local_MSE.append(x_temp)
    x = np.mean(local_MSE)
    print(x)
    if x < MSE:
        MSE = x
        temp_var_pcv = i
        
print(temp_var_pcv)





5598.68255052
4571.54899082
4742.42612548
4728.1111146
4759.94848463
4757.15035475
4795.55738595
50
49.5532670884
40.4874543379
38.3744850386
38.3725580156
39.527172598
40.6976878005
42.2712009026
150


In [8]:
print(temp_var_vol)
print(temp_var_pcv)

50
150


In [9]:
model_VOL_XG = ensemble.GradientBoostingRegressor(random_state=1,max_depth=1,n_estimators=50)
model_VOL_XG.fit(X_VOL,y_VOL)


model_PCV_XG = ensemble.GradientBoostingRegressor(random_state=1,max_depth=1,n_estimators=150)
model_PCV_XG.fit(X_PCV,y_PCV)

joblib.dump(model_VOL_XG, 'model_VOL_XG.pkl')
joblib.dump(model_PCV_XG, 'model_PCV_XG.pkl')

['model_PCV_XG.pkl']

In [10]:
from sklearn.svm import SVR
optimal_estimator = [10,50,100,150,200,250,300]
epsilon = 0

MSE = 1000000
temp_var_vol = 10000
for i in range(-5, 6):
    kf = KFold(n_splits=10)
    kf.get_n_splits(X_VOL)
    
    local_MSE = []
    for train_index, test_index in kf.split(X_VOL):
        X_train, X_test = X_VOL[train_index], X_VOL[test_index]
        y_train, y_test = y_VOL[train_index], y_VOL[test_index]
        model = SVR(C=10**i,kernel='linear',epsilon = epsilon)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        x_temp = (metrics.mean_squared_error(y_test, predictions))
        local_MSE.append(x_temp)
    x = np.mean(local_MSE)
    print(x)
    if x < MSE:
        MSE = x
        temp_var_vol = i
        
print(temp_var_vol)


MSE = 1000000
temp_var_pcv = 10000
for i in range(-5, 6):
    kf = KFold(n_splits=10)
    kf.get_n_splits(X_PCV)
    
    local_MSE = []
    for train_index, test_index in kf.split(X_PCV):
        X_train, X_test = X_PCV[train_index], X_VOL[test_index]
        y_train, y_test = y_PCV[train_index], y_PCV[test_index]
        model = SVR(C=10**i,kernel='linear',epsilon = epsilon)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        x_temp = (metrics.mean_squared_error(y_test, predictions))
        local_MSE.append(x_temp)
    x = np.mean(local_MSE)
    print(x)
    if x < MSE:
        MSE = x
        temp_var_pcv = i
        
print(temp_var_pcv)

16315.6174756
16314.7631754
16306.2189759
16222.079261
15511.1713709
11427.8282726
10339.6749014
12380.8585607
12750.3903792
12533.2879607
12533.8678999
1
67.0290678381
67.0083658413
66.8018016354
64.7948524432
50.6358694748
32.6522995634
29.993426286
30.3316997974
30.3307038414
30.3142088974
30.2547866729
1


In [10]:
print(temp_var_vol)
print(temp_var_pcv)

1
1


In [11]:
model_VOL_SVR = SVR(C=10**1,kernel='linear',epsilon = epsilon)
model_VOL_SVR.fit(X_VOL,y_VOL)


model_PCV_SVR = SVR(C=10**1,kernel='linear',epsilon = epsilon)
model_PCV_SVR.fit(X_PCV,y_PCV)

joblib.dump(model_VOL_SVR, 'model_VOL_SVR.pkl')
joblib.dump(model_PCV_SVR, 'model_PCV_SVR.pkl')

['model_PCV_SVR.pkl']

In [12]:
from sklearn import linear_model
optimal_estimator = [10,50,100,150,200,250,300]
epsilon = 0

MSE = 1000000
temp_var_vol = 10000
for i in range(-5, 6):
    kf = KFold(n_splits=10)
    kf.get_n_splits(X_VOL)
    
    local_MSE = []
    for train_index, test_index in kf.split(X_VOL):
        X_train, X_test = X_VOL[train_index], X_VOL[test_index]
        y_train, y_test = y_VOL[train_index], y_VOL[test_index]
        model = linear_model.Ridge (alpha = 10**i)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        x_temp = (metrics.mean_squared_error(y_test, predictions))
        local_MSE.append(x_temp)
    x = np.mean(local_MSE)
    print(x)
    if x < MSE:
        MSE = x
        temp_var_vol = i
        
print(temp_var_vol)


MSE = 1000000
temp_var_pcv = 10000
for i in range(-5, 6):
    kf = KFold(n_splits=10)
    kf.get_n_splits(X_PCV)
    
    local_MSE = []
    for train_index, test_index in kf.split(X_PCV):
        X_train, X_test = X_PCV[train_index], X_VOL[test_index]
        y_train, y_test = y_PCV[train_index], y_PCV[test_index]
        model = linear_model.Ridge (alpha = 10**i)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        x_temp = (metrics.mean_squared_error(y_test, predictions))
        local_MSE.append(x_temp)
    x = np.mean(local_MSE)
    print(x)
    if x < MSE:
        MSE = x
        temp_var_pcv = i
        
print(temp_var_pcv)

9787.50368879
9787.48838228
9787.33535714
9785.80909425
9770.9379477
9655.05939511
9513.10278216
10833.2250863
13481.8165701
14143.6360173
14218.8776185
1
34.0052538958
34.0051974898
34.004633587
33.9990102299
33.9443265103
33.5366584273
35.344231128
52.5495644015
61.703550613
63.1146565098
63.2646817721
0


In [12]:
print(temp_var_vol)
print(temp_var_pcv)

1
0


In [13]:
model_VOL_ridge = linear_model.Ridge (alpha = 10**1)
model_VOL_ridge.fit(X_VOL,y_VOL)


model_PCV_ridge = linear_model.Ridge (alpha = 10**0)
model_PCV_ridge.fit(X_PCV,y_PCV)

joblib.dump(model_VOL_ridge, 'model_VOL_ridge.pkl')
joblib.dump(model_PCV_ridge, 'model_PCV_ridge.pkl')

['model_PCV_ridge.pkl']